<a href="https://colab.research.google.com/github/pallavi-vishnu/Uncleaned_data/blob/main/glassdoor_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Import necessary packages.

from google.colab import files
from os import environ
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import re
sns.set()

In [42]:
# Import the dataset from kaggle

!pip install -q kaggle


In [43]:
# upload kaggle API key
uploaded = files.upload()

Saving Uncleaned_DS_jobs.csv to Uncleaned_DS_jobs (3).csv


In [44]:
# Convert the csv file into a dataframe to make it more readable.
raw_data = pd.read_csv('Uncleaned_DS_jobs.csv', index_col = 'index')
raw_data.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [45]:
# make a copy of raw data
data = raw_data.copy()

data.describe() ## Take a look at the descriptive statistics.

,Rating,Founded
count,672.000000,672.000000
mean,3.518601,1635.529762
std,1.410329,756.746640
min,-1.000000,-1.000000
25%,3.300000,1917.750000
50%,3.800000,1995.000000
75%,4.300000,2009.000000
max,5.000000,2019.000000


In [46]:
# prints information about a DataFrame including the index dtype and columns, non-null values and memory usage.

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Job Description    672 non-null    object 
 3   Rating             672 non-null    float64
 4   Company Name       672 non-null    object 
 5   Location           672 non-null    object 
 6   Headquarters       672 non-null    object 
 7   Size               672 non-null    object 
 8   Founded            672 non-null    int64  
 9   Type of ownership  672 non-null    object 
 10  Industry           672 non-null    object 
 11  Sector             672 non-null    object 
 12  Revenue            672 non-null    object 
 13  Competitors        672 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 78.8+ KB


In [61]:
# Keep only necessary columns
data = data[['Job Title', 'Salary Estimate', 'Rating', 'Company Name', 'Location', 'Size', 'Industry', 'Sector']]
data.head()

,Job Title,Salary Estimate,Rating,Company Name,Location,Size,Industry,Sector
index,,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),3.1,Healthfirst\n3.1,"New York, NY",1001 to 5000 employees,Insurance Carriers,Insurance
1,Data Scientist,$137K-$171K (Glassdoor est.),4.2,ManTech\n4.2,"Chantilly, VA",5001 to 10000 employees,Research & Development,Business Services
2,Data Scientist,$137K-$171K (Glassdoor est.),3.8,Analysis Group\n3.8,"Boston, MA",1001 to 5000 employees,Consulting,Business Services
3,Data Scientist,$137K-$171K (Glassdoor est.),3.5,INFICON\n3.5,"Newton, MA",501 to 1000 employees,Electrical & Electronic Manufacturing,Manufacturing
4,Data Scientist,$137K-$171K (Glassdoor est.),2.9,Affinity Solutions\n2.9,"New York, NY",51 to 200 employees,Advertising & Marketing,Business Services


In [62]:
# Look for missing data

data.isna().sum()

Job Title          0
Salary Estimate    0
Rating             0
Company Name       0
Location           0
Size               0
Industry           0
Sector             0
dtype: int64

In [63]:
# Take a look at the variables with object data type.

data.select_dtypes('object').head()



,Job Title,Salary Estimate,Company Name,Location,Size,Industry,Sector
index,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Healthfirst\n3.1,"New York, NY",1001 to 5000 employees,Insurance Carriers,Insurance
1,Data Scientist,$137K-$171K (Glassdoor est.),ManTech\n4.2,"Chantilly, VA",5001 to 10000 employees,Research & Development,Business Services
2,Data Scientist,$137K-$171K (Glassdoor est.),Analysis Group\n3.8,"Boston, MA",1001 to 5000 employees,Consulting,Business Services
3,Data Scientist,$137K-$171K (Glassdoor est.),INFICON\n3.5,"Newton, MA",501 to 1000 employees,Electrical & Electronic Manufacturing,Manufacturing
4,Data Scientist,$137K-$171K (Glassdoor est.),Affinity Solutions\n2.9,"New York, NY",51 to 200 employees,Advertising & Marketing,Business Services


# We can see that the data type of the column salary estimate is object.It would be convenient to have the salary estimate in numbers.

In [65]:
# Extract numeric values from the salary estimate column and convert it into list of integers
pd.options.mode.copy_on_write = True

salaries = []
for salary in data['Salary Estimate']:
  numeric = re.findall(r'\d+', salary)
  salaries.append(list(map(int, numeric)))
# Create two lists of minimum salary and maximum salary
minimum_sal = []
maximum_sal = []
for salary in salaries:
  minimum_sal.append(min(salary)*1000)
  maximum_sal.append(max(salary)*1000)

# create new columns for minimum salary and maximum salary in the raw data
data['Minimum Salary'] = minimum_sal
data['Maximum Salary'] = maximum_sal



In [66]:
# Drop the original salary estimate column from the dataset

data1 = data.drop('Salary Estimate', axis = 1)

In [52]:
data1

,Job Title,Job Description,Rating,Company Name,Location,Size,Founded,Industry,Sector,Revenue,Minimum Salary,Maximum Salary
index,,,,,,,,,,,,
0,Sr Data Scientist,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY",1001 to 5000 employees,1993,Insurance Carriers,Insurance,Unknown / Non-Applicable,137000,171000
1,Data Scientist,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA",5001 to 10000 employees,1968,Research & Development,Business Services,$1 to $2 billion (USD),137000,171000
2,Data Scientist,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA",1001 to 5000 employees,1981,Consulting,Business Services,$100 to $500 million (USD),137000,171000
3,Data Scientist,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA",501 to 1000 employees,2000,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),137000,171000
4,Data Scientist,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY",51 to 200 employees,1998,Advertising & Marketing,Business Services,Unknown / Non-Applicable,137000,171000
...,...,...,...,...,...,...,...,...,...,...,...,...
667,Data Scientist,Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT\n3.6,"Fort Lee, NJ",1001 to 5000 employees,1989,Advertising & Marketing,Business Services,Unknown / Non-Applicable,105000,167000
668,Data Scientist,Job Description\nBecome a thought leader withi...,-1.0,JKGT,"San Francisco, CA",-1,-1,-1,-1,-1,105000,167000
669,Data Scientist,Join a thriving company that is changing the w...,-1.0,AccessHope,"Irwindale, CA",-1,-1,-1,-1,-1,105000,167000


In [67]:
# Lets take a look at the Size column.

data1['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', '-1', 'Unknown'],
      dtype=object)

In [68]:
#  Make it more readable by removing the word 'employees'
data1['Size'] = data1['Size'].str.replace(' employees', '')
# Replace the word to with '-'
data1['Size'] = data1['Size'].str.replace('to', '-')


In [69]:
# Now lets take a look at the size column again
data1['Size'].value_counts()

Size
51 - 200        135
1001 - 5000     104
1 - 50           86
201 - 500        85
10000+           80
501 - 1000       77
5001 - 10000     61
-1               27
Unknown          17
Name: count, dtype: int64

**Looks like some of the organizations has a size of -1 and Unkown, which does not make any sense. Since the number of inconsistent values is 44, which is less than 10% of the size of data, we can drop the inconsistent values. Drop the inconsistent data.**

In [78]:
# Find the inconsistent data and drop the rows with inconsistent data
invalid_entries = data1[(data1['Size']=='-1') | (data1['Size']=='Unknown')].index
data_cleaned = data1.drop(invalid_entries)
data_cleaned['Size'].value_counts()
# reset the index.
data_cleaned.reset_index(drop = True)

,Job Title,Rating,Company Name,Location,Size,Industry,Sector,Minimum Salary,Maximum Salary
0,Sr Data Scientist,3.1,Healthfirst\n3.1,"New York, NY",1001 - 5000,Insurance Carriers,Insurance,137000,171000
1,Data Scientist,4.2,ManTech\n4.2,"Chantilly, VA",5001 - 10000,Research & Development,Business Services,137000,171000
2,Data Scientist,3.8,Analysis Group\n3.8,"Boston, MA",1001 - 5000,Consulting,Business Services,137000,171000
3,Data Scientist,3.5,INFICON\n3.5,"Newton, MA",501 - 1000,Electrical & Electronic Manufacturing,Manufacturing,137000,171000
4,Data Scientist,2.9,Affinity Solutions\n2.9,"New York, NY",51 - 200,Advertising & Marketing,Business Services,137000,171000
...,...,...,...,...,...,...,...,...,...
623,Data Scientist,3.8,"Criterion Systems, Inc.\n3.8","Vienna, VA",201 - 500,IT Services,Information Technology,105000,167000
624,Data Scientist,4.0,Foundation Medicine\n4.0,"Boston, MA",1001 - 5000,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,105000,167000
625,Data Scientist,3.6,TRANZACT\n3.6,"Fort Lee, NJ",1001 - 5000,Advertising & Marketing,Business Services,105000,167000
626,Data Scientist,5.0,ChaTeck Incorporated\n5.0,"San Francisco, CA",1 - 50,Advertising & Marketing,Business Services,105000,167000


In [79]:
# create a column for average salary for the different job titles
data_cleaned['Mean Salary'] = (data_cleaned['Maximum Salary']+data_cleaned['Minimum Salary'])/2
data_cleaned.head()

,Job Title,Rating,Company Name,Location,Size,Industry,Sector,Minimum Salary,Maximum Salary,Mean Salary
index,,,,,,,,,,
0,Sr Data Scientist,3.1,Healthfirst\n3.1,"New York, NY",1001 - 5000,Insurance Carriers,Insurance,137000,171000,154000.0
1,Data Scientist,4.2,ManTech\n4.2,"Chantilly, VA",5001 - 10000,Research & Development,Business Services,137000,171000,154000.0
2,Data Scientist,3.8,Analysis Group\n3.8,"Boston, MA",1001 - 5000,Consulting,Business Services,137000,171000,154000.0
3,Data Scientist,3.5,INFICON\n3.5,"Newton, MA",501 - 1000,Electrical & Electronic Manufacturing,Manufacturing,137000,171000,154000.0
4,Data Scientist,2.9,Affinity Solutions\n2.9,"New York, NY",51 - 200,Advertising & Marketing,Business Services,137000,171000,154000.0


In [80]:
# Clean up the 'Company Name' cloumn

data_cleaned['Company Name'] = data_cleaned['Company Name'].str.rstrip('\n1234567890.')
data_cleaned.head()

,Job Title,Rating,Company Name,Location,Size,Industry,Sector,Minimum Salary,Maximum Salary,Mean Salary
index,,,,,,,,,,
0,Sr Data Scientist,3.1,Healthfirst,"New York, NY",1001 - 5000,Insurance Carriers,Insurance,137000,171000,154000.0
1,Data Scientist,4.2,ManTech,"Chantilly, VA",5001 - 10000,Research & Development,Business Services,137000,171000,154000.0
2,Data Scientist,3.8,Analysis Group,"Boston, MA",1001 - 5000,Consulting,Business Services,137000,171000,154000.0
3,Data Scientist,3.5,INFICON,"Newton, MA",501 - 1000,Electrical & Electronic Manufacturing,Manufacturing,137000,171000,154000.0
4,Data Scientist,2.9,Affinity Solutions,"New York, NY",51 - 200,Advertising & Marketing,Business Services,137000,171000,154000.0


The data is cleaned and ready to use for analysis.